In [68]:
import pandas as pd
import numpy as np
import math

# read dog dataset, No is encoded as 0, Yes as 1, 
# bite refers to class 1, bark to class 0

training_data = pd.read_csv("dogs.csv") 
print(training_data)

   Heavy  Smelly  Big  Growling  Action
0      0       0    0         0       0
1      0       0    1         0       0
2      1       1    0         1       0
3      1       0    0         1       1
4      0       1    1         0       1
5      0       0    1         1       1
6      0       0    0         1       1
7      1       1    0         0       1


You can access individual columns by name to get a pandas series or convert it to a NumPy array:

In [69]:
print(training_data["Heavy"])
print(training_data["Heavy"].to_numpy())

0    0
1    0
2    1
3    1
4    0
5    0
6    0
7    1
Name: Heavy, dtype: int64
[0 0 1 1 0 0 0 1]


### What is the entropy of the target value 'Action' in the whole dataset? 
Define a function that calculates the entropy (for a list of 0/1 values):

In [70]:
#given solution
def entropy(values):
    base = len(values)
    if base <= 1:
            return 0 
    
    count0=sum([1 for i in values if i == 0])
    count1=base-count0
    p0=count0 / base
    p1=count1 / base
    
    result = - (p0*math.log2(p0) if p0 > 0 else 0) - (p1*math.log2(p1) if p1 > 0 else 0)
    return result
    

In [71]:
#own solution
def entropy(values):
        # could be empty  while trying for a split
        if values.size == 0:
            return 0
        
        ratio_ones = sum(values) / len(values)
        ratio_zeros = 1 - ratio_ones

        pd = [ratio_ones, ratio_zeros]
        if ratio_ones == 0 or ratio_zeros == 0:
            return 0
        return - sum( [ p_i * np.log2(p_i) for p_i in pd] )

Apply it to the "Action" column

In [72]:
print(entropy(training_data["Action"].to_numpy())) #Entropy of the target variable --> training entropy of the dataset before any splits are made

0.954434002924965


### Which attribute would the ID3 algorithm choose to use for the root of the tree? What is its information gain? 

Iterate through all categories, make the potential split, compare the resulting entropies and take the greedy choice. You might need:

```
# iterating the columns 
for col in data.columns: 
    print(col)
```

and to filter entries of a dataframe that satisfy a certain condition, you might want to use:
```
data[data["Heavy"] == 0] # only those entries that are not heavy
```

In [73]:
for col in training_data.columns:
    print(f'{col}:{entropy((training_data[col] == 0).to_numpy())-entropy(training_data["Action"].to_numpy())}')
    print(f'{col}:{entropy((training_data[col] == 1).to_numpy())-entropy(training_data["Action"].to_numpy())}')
#Results indicate Growling as the best Attribute

Heavy:0.0
Heavy:0.0
Smelly:0.0
Smelly:0.0
Big:0.0
Big:0.0
Growling:0.04556599707503495
Growling:0.04556599707503495
Action:0.0
Action:0.0


###  Draw the full decision tree that would be learned for this data using ID3 without pruning.

Recursively apply the splitting procedure until all nodes are leaf nodes (= pure in the target). You may use the class `Node` as a start for your implementation and attributes as needed.

In [74]:
class Node:
    
    def entropy(self, values):
        # could be empty  while trying for a split
        if values.size == 0:
            return 0
        
        ratio_ones = sum(values) / len(values)
        ratio_zeros = 1 - ratio_ones

        pd = [ratio_ones, ratio_zeros]
        if ratio_ones == 0 or ratio_zeros == 0:
            return 0
        return - sum( [ p_i * np.log2(p_i) for p_i in pd] )
    
    def __init__(self, data, ancestor_features):
        # pure if there are all ones or all zeros 
        self.pure = sum(data["Action"]) == len(data) or  sum(data["Action"]) == 0
        self.split_feature = None
        self.data = data
        self.base_entropy = self.entropy(data["Action"].to_numpy())
        self.children = None
        # since we're using categorical (binary), each features should at most be used once in any branch
        self.ancestor_features = ancestor_features 
        # You can use these ancestor_features to get a nice identation:
        # print(" "*len(self.ancestor_features) + f"Initializing with data {data}")
        
    def split(self):
        # implement the split here
        features = [col for col in self.data.columns if col not in self.ancestor_features and col != "Action"]
        max_info_gain = -1
        best_feature = None
        
        for feature in features:
            for value in self.data[feature].unique():
                subset = self.data[self.data[feature] == value]
                entropy = self.entropy(subset["Action"].to_numpy())
                info_gain = self.base_entropy - entropy
                
                if info_gain > max_info_gain:
                    max_info_gain = info_gain
                    best_feature = feature
                    best_threshold = value
        self.split_threshold = best_threshold
        if best_feature is not None:
            self.split_feature = best_feature
            self.children = []
            
            for value in self.data[best_feature].unique():
                subset = self.data[self.data[best_feature] == value]
                child = Node(subset, self.ancestor_features + [best_feature])
                self.children.append(child)
    def train(self):        
        if not self.pure:
            self.split()
            for child in self.children:
                # assume that we can get nodes pure by looking at our features
                # having exactly the same features but two different classes would not work here
                child.train()
                # now child is either pure or split into nodes that are eventually pure
# start training with a root node consisting of all data
    def predict(self, new_data):
        if self.pure:
            return self.data["Action"].iloc[0]
        else:
            # Andernfalls entscheide anhand des geteilten Merkmals und Schwellenwerts
            feature_value = new_data[self.split_feature]
            if feature_value <= self.split_threshold:
                # Wenn der Wert kleiner oder gleich dem Schwellenwert ist, gehe zum linken Kindknoten
                return self.children[0].predict(new_data)
            else:
                # Andernfalls gehe zum rechten Kindknoten
                return self.children[1].predict(new_data)
root = Node(training_data, [])
root.train()


### Explanation of the Code

Der vorliegende Python-Code implementiert einen Entscheidungsbaum (Decision Tree) für binäre Klassifikation. Hier ist eine detaillierte Erklärung des Codes:

1. Die Klasse `Node` stellt einen Knoten im Entscheidungsbaum dar. Jeder Knoten enthält eine Methode `entropy(values)`, die die Entropie für die gegebenen Klassenwerte berechnet. Die Entropie ist ein Maß für die Unreinheit der Klassenverteilung in einem Entscheidungsbaum. Niedrigere Entropie bedeutet, dass die Klassenverteilung homogener ist, während höhere Entropie auf eine gemischte Klassenverteilung hinweist.

2. Im Konstruktor `__init__(self, data, ancestor_features)` werden die Attribute eines Knotens initialisiert. `data` ist der Datensatz, der von diesem Knoten repräsentiert wird, und `ancestor_features` ist eine Liste von Merkmalen, die von den Vorfahren dieses Knotens bereits verwendet wurden. Der Knoten wird als "rein" markiert, wenn alle Datenpunkte in `data` entweder zur Klasse "1" oder zur Klasse "0" gehören. Der Basis-Entropiewert für diesen Knoten wird mit der `entropy()`-Methode berechnet.

3. Die `split()`-Methode implementiert die Spaltung des Knotens, um einen neuen Entscheidungszweig zu erstellen. Dazu wird das Merkmal mit dem höchsten Informationsgewinn (Information Gain) ausgewählt. Der Informationsgewinn wird berechnet, indem die Basisentropie des Knotens von der gewichteten Summe der entropie der nach der Spaltung entstehenden Teil-Datensätze abgezogen wird. Der beste Schwellenwert für das ausgewählte Merkmal wird ebenfalls gespeichert. Wenn ein Merkmal ausgewählt wird, werden die Kinderknoten mit neuen `Node`-Objekten erstellt und dem aktuellen Knoten als Kinder hinzugefügt.

4. Die `train()`-Methode startet den Trainingsprozess des Entscheidungsbaums, indem sie den Baum rekursiv von der Wurzel (root) aus erstellt. Wenn der aktuelle Knoten nicht "rein" ist, wird die `split()`-Methode aufgerufen, um den Knoten zu spalten und den Trainingsprozess für die Kinderknoten fortzusetzen.

5. Die `predict(new_data)`-Methode ermöglicht die Vorhersage von Klassenlabels für neue Datenpunkte basierend auf dem erstellten Entscheidungsbaum. Wenn der aktuelle Knoten "rein" ist, wird das Klassenlabel des Knotens zurückgegeben. Andernfalls wird das Merkmal des neuen Datenpunkts verwendet, um den entsprechenden Entscheidungszweig im Baum zu traversieren, indem der Schwellenwert des Merkmals mit dem Schwellenwert des aktuellen Knotens verglichen wird. Die Vorhersage wird rekursiv für die Kinderknoten fortgesetzt, bis ein "reiner" Knoten erreicht wird, dessen Klassenlabel zurückgegeben wird.

6. `train(self)`: Diese Methode wird verwendet, um den Entscheidungsbaum zu trainieren. Sie beginnt mit dem Wurzelknoten (`self`) und überprüft, ob er bereits rein ist (`self.pure`). Wenn der Knoten nicht rein ist, wird der Split durchgeführt, indem die `split()`-Methode aufgerufen wird, um das beste Merkmal und den besten Schwellenwert für den Split zu finden. Dann wird für jedes Kind des aktuellen Knotens (`self.children`) die `train()`-Methode rekursiv aufgerufen, um den Trainingsprozess für die Kinder fortzusetzen. Dadurch wird der Entscheidungsbaum weiter aufgebaut, bis alle Blattknoten rein sind oder nicht weiter gesplittet werden können.

7. `predict(self, new_data)`: Diese Methode wird verwendet, um Vorhersagen für neue Daten zu treffen. Sie beginnt beim Wurzelknoten (`self`) und überprüft, ob der Knoten rein ist (`self.pure`). Wenn ja, wird das vorhersagte Label aus dem `data`-Attribut des Knotens zurückgegeben. Andernfalls wird das Merkmal und der Schwellenwert des aktuellen Knotens verwendet, um zu entscheiden, ob der linke oder der rechte Kindknoten für die Vorhersage verwendet werden soll. Dieser Prozess wird rekursiv für das entsprechende Kind fortgesetzt, bis ein Blattknoten erreicht wird, dessen vorhergesagtes Label zurückgegeben wird.

8. Schließlich wird ein `Node`-Objekt namens `root` erstellt, das mit den Trainingsdaten und einer leeren Liste von Vorgänger-Merkmalen initialisiert wird. Die `train()`-Methode wird auf `root` aufgerufen, um den Entscheidungsbaum zu trainieren und den Baum aufzubauen.

### Test on three new dogs
Here's the test set, evaluate the predicted "Action" classes using your implementation and compare them to the true "Action" classes.

In [75]:
temp_data = np.array(pd.read_csv("dogs_test.csv"))
print(temp_data)

[[1 0 1 1 0]
 [0 0 0 1 0]
 [1 1 0 0 1]]


In [76]:

test_data = [{"Heavy":1,"Smelly":0,"Big":1,"Growling":1}, {"Heavy": 0, "Smelly": 0, "Big": 0, "Growling": 1},{"Heavy": 1, "Smelly": 1, "Big": 0, "Growling": 0}]
for data_point in test_data:
    print("Prediction:"+ str(root.predict(data_point)))

Prediction:0
Prediction:0
Prediction:1


### Repeat with the Gini coefficient
Just alter your previous implementations 

In [77]:
#defining and testing gini function
def gini(values):
    base=len(values)
    if values.size == 0:
        return 0
    count0=sum([1 for i in values if i == 0])
    count1=base-count0
    p0=count0 / base
    p1=count1 / base
    result = 1-(p0**2+p1**2)
    return result
    
print(gini(training_data["Action"].to_numpy())) #0.46875 confirmed by calculator
    

0.46875


In [78]:
#rewriting class with gini instead of entropy
class Node:
    
    def gini(self,values):
        base=len(values)
        if values.size == 0:
            return 0
        count0=sum([1 for i in values if i == 0])
        count1=base-count0
        p0=count0 / base
        p1=count1 / base
        result = 1-(p0**2+p1**2)
        return result
    
    def __init__(self, data, ancestor_features):
        # pure if there are all ones or all zeros 
        self.pure = sum(data["Action"]) == len(data) or  sum(data["Action"]) == 0
        self.split_feature = None
        self.data = data
        self.base_gini = self.gini(data["Action"].to_numpy())
        self.children = None
        # since we're using categorical (binary), each features should at most be used once in any branch
        self.ancestor_features = ancestor_features 
        # You can use these ancestor_features to get a nice identation:
        print(" "*len(self.ancestor_features) + f"{data}")
        
    def split(self):
        # implement the split here
        features = [col for col in self.data.columns if col not in self.ancestor_features and col != "Action"]
        max_info_gain = -1
        best_feature = None
        
        for feature in features:
            for value in self.data[feature].unique():
                subset = self.data[self.data[feature] == value]
                gini = self.gini(subset["Action"].to_numpy())
                info_gain = self.base_gini - gini
                
                if info_gain > max_info_gain:
                    max_info_gain = info_gain
                    best_feature = feature
                    best_threshold = value
        self.split_threshold = best_threshold
        if best_feature is not None:
            self.split_feature = best_feature
            self.children = []
            
            for value in self.data[best_feature].unique():
                subset = self.data[self.data[best_feature] == value]
                child = Node(subset, self.ancestor_features + [best_feature])
                self.children.append(child)
    def train(self):        
        if not self.pure:
            self.split()
            for child in self.children:
                # assume that we can get nodes pure by looking at our features
                # having exactly the same features but two different classes would not work here
                child.train()
                # now child is either pure or split into nodes that are eventually pure
# start training with a root node consisting of all data
    def predict(self, new_data):
        if self.pure:
            return self.data["Action"].iloc[0]
        else:
            # Andernfalls entscheide anhand des geteilten Merkmals und Schwellenwerts
            feature_value = new_data[self.split_feature]
            if feature_value <= self.split_threshold:
                # Wenn der Wert kleiner oder gleich dem Schwellenwert ist, gehe zum linken Kindknoten
                return self.children[0].predict(new_data)
            else:
                # Andernfalls gehe zum rechten Kindknoten
                return self.children[1].predict(new_data)
root_gini = Node(training_data, [])
root_gini.train()


   Heavy  Smelly  Big  Growling  Action
0      0       0    0         0       0
1      0       0    1         0       0
2      1       1    0         1       0
3      1       0    0         1       1
4      0       1    1         0       1
5      0       0    1         1       1
6      0       0    0         1       1
7      1       1    0         0       1
    Heavy  Smelly  Big  Growling  Action
0      0       0    0         0       0
1      0       0    1         0       0
4      0       1    1         0       1
7      1       1    0         0       1
    Heavy  Smelly  Big  Growling  Action
2      1       1    0         1       0
3      1       0    0         1       1
5      0       0    1         1       1
6      0       0    0         1       1
     Heavy  Smelly  Big  Growling  Action
0      0       0    0         0       0
1      0       0    1         0       0
4      0       1    1         0       1
     Heavy  Smelly  Big  Growling  Action
7      1       1    0         0   

In [79]:
for data_point in test_data:
    print("Prediction:"+ str(root_gini.predict(data_point)))


Prediction:0
Prediction:0
Prediction:1
